In [ ]:
import polygon_ECT as poly
import numpy as np
from shape_reader import ShapeReader
import torch
# Read in shapes
s1=ShapeReader.shape_from_file('data/polyhedron.off',prepare=True)
s2=ShapeReader.shape_from_file('data/polyhedron.off',prepare=False)

In [ ]:
#Rotate one of the shapes
import polygon_ECT as polyx
from scipy.spatial.transform import Rotation as R
tmp=R.random(random_state=100)
rotmatrix=tmp.as_matrix()
s2.V=np.matmul(s2.V,rotmatrix)
s2.prepare()
# Compute the proto-transforms:
ECT1=poly.return_ECT(s1)
ECT2=poly.return_ECT(s2)

In [ ]:
target_inner_product=poly.ECT_distance_s(ECT1)
print(target_inner_product)

In [ ]:
import multiprocessing
import itertools

In [ ]:
def rotate_ECT(ECT,matrix):
    ECT_tmp=list()
    for i in range(len(ECT)):
        tmp=list()
        tmp.append(ECT[i][0])
        tmp.append(np.matmul(ECT[i][1],matrix))
        tmp.append(np.matmul(ECT[i][2],matrix))
        tmp.append(ECT[i][3])
        tmp.append(np.matmul(ECT[i][4],matrix))
        ECT_tmp.append(tmp)
    return(ECT_tmp)
def penalty(angles):
    rotation=R.from_euler('xyz',angles).as_matrix()
    ECT_tmp=rotate_ECT(ECT1,rotation)
    distance=poly.ECT_distance_d(ECT_tmp,ECT2)
    return(distance,angles)
def SO3_distance(angles):
    matrix=R.from_euler('xyz',angles).as_matrix()
    diff=np.matmul(rotmatrix,matrix.transpose())
    upstairs=np.sum(np.diag(diff))-1
    return(np.arccos(upstairs/2))

In [ ]:
# Setup for initial alignment as described in the manuscript
import math
gridsize=8
gridsize2=4
angles1=np.linspace(0,2*math.pi,gridsize+1)[1:]
angles2=np.linspace(0,2*math.pi,gridsize+1)[1:]
angles3=np.linspace(0,math.pi,gridsize2+1)
lista=[p for p in itertools.product(angles1,angles2,angles3)]

In [ ]:
# Parallelize (16 cor
pool = multiprocessing.Pool(16)
out1,out2 = zip(*pool.map(penalty, lista))

In [ ]:
# Subsequent iterations
iters=10
for i in range(iters):
    results=np.array(out1).reshape(-1)
    indices=np.argsort(results)
    best_ones=indices[::-1] 
    print('Inner product:')
    print(results[best_ones[0]])
    print('Estimate:')
    print(np.round(out2[best_ones[0]],5))
    print('Distance to truth:')
    print(np.round(SO3_distance(out2[best_ones[0]]),5))
    stepsize=(angles1[1]-angles1[0])
    la1=out2[best_ones[0]][0]-stepsize
    ua1=out2[best_ones[0]][0]+stepsize
    la2=out2[best_ones[0]][1]-stepsize
    ua2=out2[best_ones[0]][1]+stepsize
    la3=out2[best_ones[0]][2]-stepsize
    ua3=out2[best_ones[0]][2]+stepsize
    angles1=np.linspace(la1,ua1,4)
    angles2=np.linspace(la2,ua2,4)
    angles3=np.linspace(la3,ua3,4)
    lista=[p for p in itertools.product(angles1,angles2,angles3)]
    pool = multiprocessing.Pool(16)
    out1,out2 = zip(*pool.map(penalty, lista))